In [1]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  
EXPERIMENT_NAME = "/Users/diegooctavioperez21@gmail.com/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-02.parquet')

In [5]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [6]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [9]:
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [10]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [11]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

In [21]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-23 16:55:32,679] A new study created in memory with name: no-name-5dee745d-d51f-4088-844d-511a22c6e705


[0]	validation-rmse:5.98960
[1]	validation-rmse:5.92413
[2]	validation-rmse:5.92287
[3]	validation-rmse:5.92324
[4]	validation-rmse:5.92483
[5]	validation-rmse:5.92489
[6]	validation-rmse:5.92498
[7]	validation-rmse:5.92343
[8]	validation-rmse:5.92347
[9]	validation-rmse:5.92467
[10]	validation-rmse:5.92456
[11]	validation-rmse:5.92460


/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [16:55:37] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/23 16:55:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [16:55:39] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-23 16:55:40,943] Trial 0 finished with value: 5.924581902007077 and parameters: {'max_

🏃 View run angry-jay-533 at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722/runs/351395e99c514181bac3d63a6d870bed
🧪 View experiment at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722
[0]	validation-rmse:8.79510
[1]	validation-rmse:8.50586
[2]	validation-rmse:8.24215
[3]	validation-rmse:8.00493
[4]	validation-rmse:7.78830
[5]	validation-rmse:7.59575
[6]	validation-rmse:7.41983
[7]	validation-rmse:7.26129
[8]	validation-rmse:7.11270
[9]	validation-rmse:6.98583
[10]	validation-rmse:6.87056
[11]	validation-rmse:6.76533
[12]	validation-rmse:6.67029
[13]	validation-rmse:6.58378
[14]	validation-rmse:6.50773
[15]	validation-rmse:6.43768
[16]	validation-rmse:6.37633
[17]	validation-rmse:6.32116
[18]	validation-rmse:6.27142
[19]	validation-rmse:6.22765
[20]	validation-rmse:6.18963
[21]	validation-rmse:6.15802
[22]	validation-rmse:6.12794
[23]	validation-rmse:6.10295
[24]	validation-rmse:6.07940
[25]	validation-rmse:6.05863
[26]	vali

/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [16:55:46] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/23 16:55:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [16:55:47] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-23 16:55:49,809] Trial 1 finished with value: 5.902611653559419 and parameters: {'max_

...

🏃 View run puzzled-ram-480 at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722/runs/6374191fb5c4440c8155654b9c754e10
🧪 View experiment at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722
[0]	validation-rmse:8.41579
[1]	validation-rmse:7.85713
[2]	validation-rmse:7.42170
[3]	validation-rmse:7.07938
[4]	validation-rmse:6.80816
[5]	validation-rmse:6.60651
[6]	validation-rmse:6.45200
[7]	validation-rmse:6.32855
[8]	validation-rmse:6.24433
[9]	validation-rmse:6.18085
[10]	validation-rmse:6.11653
[11]	validation-rmse:6.07718
[12]	validation-rmse:6.04368
[13]	validation-rmse:6.02265
[14]	validation-rmse:6.01240
[15]	validation-rmse:6.00241
[16]	validation-rmse:5.99576
[17]	validation-rmse:5.99117
[18]	validation-rmse:5.98779
[19]	validation-rmse:5.98629
[20]	validation-rmse:5.98758
[21]	validation-rmse:5.98578
[22]	validation-rmse:5.98501
[23]	validation-rmse:5.98725
[24]	validation-rmse:5.98845
[25]	validation-rmse:5.99028
[26]	va

/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [16:57:09] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/23 16:57:11 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [16:57:11] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-23 16:57:16,181] Trial 9 finished with value: 5.999083279452748 and parameters: {'max_

🏃 View run nimble-kite-318 at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722/runs/1153b0a7b74541a2b84cd34a05a89bda
🧪 View experiment at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722
[0]	validation-rmse:7.96125
[1]	validation-rmse:7.19956
[2]	validation-rmse:6.69631
[3]	validation-rmse:6.37886
[4]	validation-rmse:6.17661
[5]	validation-rmse:6.05039
[6]	validation-rmse:5.98463
[7]	validation-rmse:5.93473
[8]	validation-rmse:5.90786
[9]	validation-rmse:5.89293
[10]	validation-rmse:5.88940
[11]	validation-rmse:5.88270
[12]	validation-rmse:5.87865
[13]	validation-rmse:5.87724
[14]	validation-rmse:5.87452
[15]	validation-rmse:5.87431
[16]	validation-rmse:5.87284
[17]	validation-rmse:5.87217
[18]	validation-rmse:5.87229
[19]	validation-rmse:5.87087
[20]	validation-rmse:5.87091
[21]	validation-rmse:5.87085
[22]	validation-rmse:5.87091
[23]	validation-rmse:5.86997
[24]	validation-rmse:5.86922
[25]	validation-rmse:5.86934
[26]	va

/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [16:57:21] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/23 16:57:22 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [16:57:22] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run XGBoost Hyperparameter Optimization (Optuna) at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722/runs/863dbc661684448387c87c1b40bcef19
🧪 View experiment at: https://dbc-8a29a60a-e19d.cloud.databricks.com/ml/experiments/3813900182186722


In [13]:
model_name = "workspace.default.nyc-taxi-model"

In [14]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="workspace.default.nyc-taxi-model"
)

Successfully registered model 'workspace.default.nyc-taxi-model'.
2025/10/23 16:54:26 WARNING mlflow.tracking._model_registry.fluent: Run with id 595b763b773c4594ad67dfdc872fd14c has no artifacts at artifact path 'model', registering model based on models:/m-bb782915966349f2ab067cad144bce04 instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '1' of model 'workspace.default.nyc-taxi-model'.


In [15]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

# Obtener el mejor run
if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

🏆 Champion Run encontrado:
Run ID: 4eb500fc50574433a9a066dbd0996187
RMSE: 5.873624736518469
Params: {'early_stopping_rounds': '10', 'custom_metric': 'None', 'learning_rate': '0.9136840519292246', 'max_depth': '5', 'maximize': 'None', 'min_child_weight': '0.7613210498541186', 'num_boost_round': '100', 'objective': 'reg:squarederror', 'reg_alpha': '0.18820387978911576', 'reg_lambda': '0.007166739666045858', 'seed': '42', 'verbose_eval': 'True'}


In [16]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=model_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/23 16:54:34 WARNING mlflow.tracking._model_registry.fluent: Run with id 4eb500fc50574433a9a066dbd0996187 has no artifacts at artifact path 'model', registering model based on models:/m-cdc87c46fe7441ae8a499bc42cde3cb7 instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '2' of model 'workspace.default.nyc-taxi-model'.


In [17]:
from mlflow import MlflowClient

client = MlflowClient()

In [18]:
model_version = result.version
new_alias = "Champion"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [20]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1761260077156, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 2 was transitioned to Champion on 2025-10-23 16:54:53.712830', last_updated_timestamp=1761260094017, metrics=[<Metric: dataset_digest='', dataset_name='', key='best_iteration', model_id='m-cdc87c46fe7441ae8a499bc42cde3cb7', run_id='4eb500fc50574433a9a066dbd0996187', step=0, timestamp=1761260032630, value=20.0>,
 <Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-cdc87c46fe7441ae8a499bc42cde3cb7', run_id='4eb500fc50574433a9a066dbd0996187', step=0, timestamp=1761260034157, value=5.873624736518469>,
 <Metric: dataset_digest='', dataset_name='', key='stopped_iteration', model_id='m-cdc87c46fe7441ae8a499bc42cde3cb7', run_id='4eb500fc50574433a9a066dbd0996187', s

In [19]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Champion"

champion_version = mlflow.pyfunc.load_model(model_version_uri)
champion_version.predict(X_val)

/Users/octavioperezurbina/Documents/iteso/proyecto_cd/nyc-taxi-predictions/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [16:54:51] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


array([ 4.7416825, 24.05205  , 29.134676 , ..., 21.56244  , 13.990461 ,
       24.05205  ], shape=(44218,), dtype=float32)